In [1]:
import pygsti
from pygsti.circuits import Circuit
import numpy as np
from scipy.linalg import expm, sqrtm
# import ordered dict
from collections import OrderedDict

In [2]:
from pygsti.modelpacks import smq1Q_XYI
from pygsti.modelmembers.operations import create_from_unitary_mx
from scipy.linalg import expm, logm
from pygsti.tools.jamiolkowski import jamiolkowski_iso
from pygsti.tools.basistools import change_basis
from pygsti.modelmembers.operations import FullArbitraryOp


In [3]:
def PauliMatrix(i):
    if i == 0:
        return np.array([[1, 0], [0, 1]])
    elif i == 1:
        return np.array([[0, 1], [1, 0]])
    elif i == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif i == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError("i must be 0, 1, 2, or 3.")

def PauliTensor(i, j):
    return np.kron(PauliMatrix(i), PauliMatrix(j))

In [4]:
single_qubit_projectors = {
    '0' : (np.eye(2) + PauliMatrix(3))/2,
    '1' : (np.eye(2) - PauliMatrix(3))/2,
    '+' : (np.eye(2) + PauliMatrix(1))/2, 
    '-' : (np.eye(2) - PauliMatrix(1))/2,
    'r' : (np.eye(2) + PauliMatrix(2))/2,
    'l' : (np.eye(2) - PauliMatrix(2))/2
}

In [5]:
def prep_name_to_gates(prep_name):
    if prep_name == '0':
        return []
    elif prep_name == '1':
        return ['Gx']
    elif prep_name == '+':
        return ['Gy']
    elif prep_name == '-':
        return ['Gy']*3
    elif prep_name == 'r':
        return ['Gx']*3
    elif prep_name == 'l':
        return ['Gx']
    else:
        raise ValueError("prep_name must be 0, 1, +, -, r, or l.")

def meas_name_to_gates(meas_name):
    if meas_name == '0':
        return []
    elif meas_name == '+':
        return ['Gy']*3
    elif meas_name == 'r':
        return ['Gx']
    else:
        raise ValueError("meas_name must be 0, +, or r.")


In [6]:
from pygsti.modelmembers.povms import UnconstrainedPOVM
from pygsti.modelmembers.operations import StaticArbitraryOp

In [144]:
class ProcessTomography1Q:
    def __init__(self, qid, process_name='u'):
        self.process_name = process_name
        self.qid = qid
        self.fiducial_pairs = self._create_fiducial_pairs()
        self.edesign_pygsti = self._make_pygsti_circuits(self.process_name)

    def _create_fiducial_pairs(self):
        all_fiducials = OrderedDict()
        for prep_fiducial in ['0', '1', '+', '-', 'r', 'l']:
            for meas_fiducial in ['0', '1', '+', '-', 'r', 'l']:
                all_fiducials[prep_fiducial+'.'+meas_fiducial] = 2*np.kron(single_qubit_projectors[prep_fiducial].T, single_qubit_projectors[meas_fiducial])
        return all_fiducials

    def design_rvecs(self):
        rvecs = dict()
        for lbl, fid in self.fiducial_pairs.items():
            rvecs[lbl] = fid.flatten()
        return rvecs
    
    def design_matrix(self):
        rvecs = self.design_rvecs()
        return np.array(list(rvecs.values()))
    
    def format_data_dict(self, data):
        data_dict = {}
        for key, value in data.items():
            prep, meas = key.split('.')
            if meas == '0':
                data_dict[key] = value[0]
                data_dict[prep + '.1'] = value[1]
            elif meas == '+':
                data_dict[key] = value[0]
                data_dict[prep + '.-'] = value[1]
            elif meas == 'r':
                data_dict[key] = value[0]
                data_dict[prep + '.l'] = value[1]
        return data_dict
    
    def format_dvec(self, data):
        data_dict = self.format_data_dict(data)
        dvec = np.array([data_dict[key] for key in self.fiducial_pairs.keys()])
    
    def _make_pygsti_circuits(self, gate_label):
        # create a list of circuits
        circuits = {}
        for prep in ['0', '1', '+', '-', 'r', 'l']:
            for meas in ['0', '+', 'r']:
                prep_gates = prep_name_to_gates(prep)
                meas_gates = meas_name_to_gates(meas)
                key = prep + '.' + meas
                circuits[key] = Circuit(prep_gates + ['Gu'] + meas_gates)
        return circuits
    
    def get_pygsti_model(self, gate_label, gate_pmat, gate_pmat_basis='std'):
        # create a pygsti model
        model = pygsti.models.ExplicitOpModel([self.qid],'std')
        # add the gate
        model.operations[f'G{gate_label}'] = StaticArbitraryOp(gate_pmat, 'std', 'densitymx')
        # add X and Y pi/2 rotations
        Xpi2 = expm(-1j*np.pi/4*PauliMatrix(1))
        Ypi2 = expm(-1j*np.pi/4*PauliMatrix(2))
        model.operations['Gx'] = StaticArbitraryOp(create_from_unitary_mx(Xpi2, 'static', 'std').to_dense(), 'std', 'densitymx')
        model.operations['Gy'] = StaticArbitraryOp(create_from_unitary_mx(Ypi2, 'static', 'std').to_dense(), 'std', 'densitymx')
        # add the SPAM
        model['rho0'] = [1, 0, 0, 0]
        model['Mdefault'] = UnconstrainedPOVM({'0': [1, 0, 0, 0], '1': [ 0, 1, 0, 0]}, evotype='densitymx')
        return model


In [145]:
create_from_unitary_mx(expm(-1j*np.pi/4*PauliMatrix(2)), 'static', 'std').to_dense()

array([[ 0.5+0.j, -0.5-0.j, -0.5+0.j,  0.5+0.j],
       [ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
       [ 0.5+0.j, -0.5-0.j,  0.5+0.j, -0.5-0.j],
       [ 0.5+0.j,  0.5+0.j,  0.5+0.j,  0.5+0.j]])

In [146]:
tomo = ProcessTomography1Q(0)
tomo.design_matrix().shape

(36, 16)

In [147]:
def random_unitary():
    phases = np.random.rand(3)*2*np.pi
    U = expm(-(1j/2)*(phases[0]*PauliMatrix(1) + phases[1]*PauliMatrix(2) + phases[2]*PauliMatrix(3)))
    return U

U = random_unitary()
Gu = create_from_unitary_mx(U, 'static', 'std')

In [148]:
model = tomo.get_pygsti_model('u', Gu.to_dense())

In [149]:
Gu.to_dense()

array([[ 0.77998305+0.j        , -0.3267961 +0.25458555j,
        -0.3267961 -0.25458555j,  0.22001695+0.j        ],
       [ 0.39095054+0.13699331j,  0.73920499-0.24889663j,
        -0.1190854 -0.18500305j, -0.39095054-0.13699331j],
       [ 0.39095054-0.13699331j, -0.1190854 +0.18500305j,
         0.73920499+0.24889663j, -0.39095054+0.13699331j],
       [ 0.22001695+0.j        ,  0.3267961 -0.25458555j,
         0.3267961 +0.25458555j,  0.77998305+0.j        ]])

In [150]:
model['Gu'].to_dense()

array([[ 0.77998305+0.j        , -0.3267961 +0.25458555j,
        -0.3267961 -0.25458555j,  0.22001695+0.j        ],
       [ 0.39095054+0.13699331j,  0.73920499-0.24889663j,
        -0.1190854 -0.18500305j, -0.39095054-0.13699331j],
       [ 0.39095054-0.13699331j, -0.1190854 +0.18500305j,
         0.73920499+0.24889663j, -0.39095054+0.13699331j],
       [ 0.22001695+0.j        ,  0.3267961 -0.25458555j,
         0.3267961 +0.25458555j,  0.77998305+0.j        ]])

In [151]:
def simulate_edesign(model, edesign, num_shots):
    emperical_data = {}
    for lbl, circ in edesign.items():
        probs = model.probabilities(circ)
        p0 = probs['0']
        p1 = probs['1']
        pvec = np.array([p0, p1])
        pvec = np.clip(pvec, 0, 1)
        emperical_data[lbl] = np.random.multinomial(num_shots, pvec)/num_shots
    return emperical_data


In [152]:
edesign = tomo.edesign_pygsti

In [153]:
edesign

{'0.0': Circuit(Gu),
 '0.+': Circuit(GuGyGyGy),
 '0.r': Circuit(GuGx),
 '1.0': Circuit(GxGu),
 '1.+': Circuit(GxGuGyGyGy),
 '1.r': Circuit(GxGuGx),
 '+.0': Circuit(GyGu),
 '+.+': Circuit(GyGuGyGyGy),
 '+.r': Circuit(GyGuGx),
 '-.0': Circuit(GyGyGyGu),
 '-.+': Circuit(GyGyGyGuGyGyGy),
 '-.r': Circuit(GyGyGyGuGx),
 'r.0': Circuit(GxGxGxGu),
 'r.+': Circuit(GxGxGxGuGyGyGy),
 'r.r': Circuit(GxGxGxGuGx),
 'l.0': Circuit(GxGu),
 'l.+': Circuit(GxGuGyGyGy),
 'l.r': Circuit(GxGuGx)}

In [154]:
data = simulate_edesign(model, edesign, 1000)

/home/jpmarceaux/anaconda3/envs/ErrorGenerators/lib/python3.10/site-packages/pygsti/forwardsims/matrixforwardsim.py:746: ComplexWarning: Casting complex values to real discards the imaginary part
  prodCache[iDest] = gate / nG


In [155]:
data

{'0.0': array([0.779, 0.221]),
 '0.+': array([0.878, 0.122]),
 '0.r': array([0.508, 0.492]),
 '1.0': array([0.519, 0.481]),
 '1.+': array([0.496, 0.504]),
 '1.r': array([0.505, 0.495]),
 '+.0': array([0.159, 0.841]),
 '+.+': array([0.815, 0.185]),
 '+.r': array([0.519, 0.481]),
 '-.0': array([0.838, 0.162]),
 '-.+': array([0.21, 0.79]),
 '-.r': array([0.489, 0.511]),
 'r.0': array([0.513, 0.487]),
 'r.+': array([0.501, 0.499]),
 'r.r': array([0.496, 0.504]),
 'l.0': array([0.505, 0.495]),
 'l.+': array([0.51, 0.49]),
 'l.r': array([0.477, 0.523])}

In [156]:
formatted_data = tomo.format_data_dict(data)

In [157]:
formatted_data

{'0.0': 0.779,
 '0.1': 0.221,
 '0.+': 0.878,
 '0.-': 0.122,
 '0.r': 0.508,
 '0.l': 0.492,
 '1.0': 0.519,
 '1.1': 0.481,
 '1.+': 0.496,
 '1.-': 0.504,
 '1.r': 0.505,
 '1.l': 0.495,
 '+.0': 0.159,
 '+.1': 0.841,
 '+.+': 0.815,
 '+.-': 0.185,
 '+.r': 0.519,
 '+.l': 0.481,
 '-.0': 0.838,
 '-.1': 0.162,
 '-.+': 0.21,
 '-.-': 0.79,
 '-.r': 0.489,
 '-.l': 0.511,
 'r.0': 0.513,
 'r.1': 0.487,
 'r.+': 0.501,
 'r.-': 0.499,
 'r.r': 0.496,
 'r.l': 0.504,
 'l.0': 0.505,
 'l.1': 0.495,
 'l.+': 0.51,
 'l.-': 0.49,
 'l.r': 0.477,
 'l.l': 0.523}